In [ ]:
import torch 
from torch.utils.data import Dataset, DataLoader, Subset 
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
from datasets import load_dataset 
import random 

# Set random seed for reproducibility 
random.seed(42)
torch.manual_seed(42) 

# Load SST-2 dataset 
dataset = load_dataset("sst2") 

# Initialize tokenizer 
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") 

class SST2Dataset(Dataset):
    def __init__(self, split):
        self.data = dataset[split]
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]["sentence"]
        label = self.data[idx]["label"]
        encoding = tokenizer(text, return_tensors="pt", padding="max_length", 
                             truncation=True, max_length=128)
        return {
            "input_ids": encoding["input_ids"].squeeze(), 
            "attention_mask": encoding["attention_mask"].squeeze(), 
            "labels": torch.tensor(label)
        }
        
# Create full datasets 
full_train_dataset = SST2Dataset("train")
full_eval_dataset = SST2Dataset("validation")

# Randomly sample smaller subsets 
train_indices = random.sample(range(len(full_train_dataset)), 1600)
eval_indices = random.sample(range(len(full_eval_dataset)), 32)

train_dataset = Subset(full_train_dataset, train_indices)
eval_dataset = Subset(full_eval_dataset, eval_indices)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16)

In [ ]:
import torch.nn as nn 
import torch.nn.functional as F 

class DPOModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits 
    
def dpo_loss(logits, labels, beta=0.1):
    chosen_logits = logits[labels == 1]
    rejected_logits = logits[labels == 0]
    
    # Ensure we have both positive and negative samples 
    if chosen_logits.size(0) == 0 or rejected_logits.size(0) == 0:
        return torch.tensor(0.0, device=logits.device)
    
    # Compute rewards 
    chosen_rewards = chosen_logits.gather(1, torch.argmax(chosen_logits, dim=1).unsqueeze(1)).squeeze()
    rejected_rewards = rejected_logits.gather(1, torch.argmax(rejected_logits, dim=1).unsqueeze(1)).squeeze()
    
    # Compute pairwise differences 
    chosen_rewards = chosen_rewards.unsqueeze(1)
    rejected_rewards = rejected_rewards.unsqueeze(0)
    diff = chosen_rewards - rejected_rewards 
    
    # Compute loss 
    loss = -F.logsigmoid(beta * diff).mean() 
    return loss 